# Background Scan Operation

How to run update operations on a namespace in background.
This notebook requires Aerospike datbase running locally and that Java kernel has been installed. Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) for additional details and the docker container.

## Ensure database is running
This notebook requires that Aerospike datbase is running.

In [2]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);
%sh asd

## Download Aerospike client from POM

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
</dependencies>

## Initialize the client and populate test data

Initialize the client and connect to the cluster. The configuration is for Aerospike database running on port 3000 of localhost which is the default. Modify config if your environment is different (Aerospike database running on a different host or different port).


In [ ]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

String Namespace = "test";
String Set = "background-scan-update";

// add records with keys "id-1" to "id-10" and 
//    bins bin1 (integer values 1-10) and bin2 (integer values 1000-1010).
WritePolicy wpolicy = new WritePolicy();
wpolicy.sendKey = true;
for (int i = 1; i <= 10; i++) {
    Key key = new Key(Namespace, Set, "id-"+i);
    Bin bin1 = new Bin(new String("bin1"), i);
    Bin bin2 = new Bin(new String("bin2"), 1000+i);
    client.put(wpolicy, key, bin1, bin2);
}
System.out.format("Test data popuated.\n");;

# Scan and Update Namespace
Multi-record updates and deletes can be done in two ways:

1. List of bin updates: A list of update operations can be specified, each operating on a bin. The list can be specified in two ways:
   1. As part of the Statement object where the query (index) filter is also specified.
   <pre>
   Statement::setOps(Operation[] ops]
   AerospikeClient::execute(WritePolicy policy, Statement statement)
   </pre>
   2. As an operation list parameter in the execute call.
   <pre>
   AerospikeClient::execute(WritePolicy policy, Statement statement, Operation[] ops)
   </pre>
2. User Defined Function (UDF): Record-oriented UDFs implement arbitrary logic in a Lua function that is registered with the server and invoked through an API call.
   <pre>
   ExecuteTask execute(WritePolicy policy, Statement statement, String packageName, String functionName,
                                 Value... functionArgs)
   </pre>

In [6]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.query.Statement;
import com.aerospike.client.Operation;
import com.aerospike.client.task.ExecuteTask;

int POLL_INTERVAL_MS = 3000;
int WAIT_TILL_COMPLETE_MS = 100000;

enum ExecOption{STMT, EXEC_ARG, UDF};

WritePolicy wpolicy = new WritePolicy();
wpolicy.sendKey = true;
wpolicy.maxRetries = 0;
// specify TTL here
wpolicy.expiration = -2;

void scanAndUpdate(String seedNodeIp, String namespace, Operation[] ops, ExecOption option) {
    System.out.format("Performing scan-and-update on namespace %s with option %s.\n", namespace, option);
    AerospikeClient client = new AerospikeClient(seedNodeIp, 3000);
    Statement stmt = new Statement();
    ExecuteTask task;
    stmt.setNamespace(namespace);
    switch(option) {
        case STMT:
            stmt.setOperations(ops);
            task = client.execute(wpolicy, stmt); 
            task.waitTillComplete(POLL_INTERVAL_MS, WAIT_TILL_COMPLETE_MS);
            break;
      case EXEC_ARG:
            task = client.execute(wpolicy, stmt, ops); 
            task.waitTillComplete(POLL_INTERVAL_MS, WAIT_TILL_COMPLETE_MS);
            break;
      case UDF:
            // plug in UDFModule, UDFFunction (assumes you have registered the module separately)
            //task = client.execute(wpolicy, stmt, "UDFModule", "UDFFunction"); 
            //task.waitTillComplete(POLL_INTERVAL_MS, WAIT_TILL_COMPLETE_MS);
            break;
      default:
            System.out.format("Error: wrong ExecOption\n");
    }
    client.close();
    System.out.format("Scan-and-update done.\n");;
}

In [7]:
String seedNode = "127.0.0.1";
String namespace = "test";

Operation ops[] = { Operation.touch() };
scanAndUpdate(seedNode, namespace, ops, ExecOption.STMT);
scanAndUpdate(seedNode, namespace, ops, ExecOption.EXEC_ARG);
scanAndUpdate(seedNode, namespace, ops, ExecOption.UDF);

Performing scan-and-update on namespace test with option STMT.
Scan-and-update done.
Performing scan-and-update on namespace test with option EXEC_ARG.
Scan-and-update done.
Performing scan-and-update on namespace test with option UDF.
Scan-and-update done.


## Clean up
Finally close the client connection.

In [ ]:
client.truncate(null, Namespace, null, null);
client.close();
System.out.println("Removed tutorial data and closed server connection.");

## Next steps

Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) to run additional Aerospike notebooks. To run a different notebook, download the notebook from the repo to your local machine, and then click on File->Open, and select Upload.